# Timer

In [1]:
from datetime import datetime as dt

start_time = "25.11.2022, 11:15"
start_time = dt.strptime(start_time, "%d.%m.%Y, %H:%M")

while True:
    if start_time < dt.now():
        break
    else:
        sleep(10)

# Parameter optimization

In [1]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

ttype = 'buy'
pattern = ['STOCH', 'RSI', 'Trend']
indicator_list = pattern
indicator_list_higher = ['Trend']

work_timeframe = '15m'
higher_timeframe = '1h'
opt_limit = 2000
load = False

print(f'Timeframe is {work_timeframe}/{higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Indicator_list'] = indicator_list
configs['Higher_TF_indicator_list'] = indicator_list_higher
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {
                'RSI': {
                        'timeperiod': [14], 
                        'low_bound': [35],
                        'vol_window': [48], # [6, 12, 24, 48, 72, 96],
                        'vol_q_high': [0.2], # np.arange(0.1, 1, 0.05),
                        'vol_q_low': [0.8], # np.arange(0.1, 1, 0.05),
                        'first_candle': [0.8], # np.arange(0.5, 1, 0.05),
                        'second_candle': [0.5] # np.arange(0.5, 1, 0.05)
                       },
                'STOCH': {
                          'fastk_period': [9],
                          'slowk_period': [7],
                          'slowd_period': [3], 
                          'low_bound': [25]
                        },
                'Trend': {
                          'timeperiod': np.arange(2, 102, 2), 
                          'low_bound': [0]
                        }
             }

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')


Timeframe is 15m/1h, trade type is buy
Number of combinations is 50


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [05:18<00:00,  6.36s/it]


# Check local statistics

In [20]:
# candles
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_rank', ascending=False)
stat

,pattern,RSI_timeperiod,RSI_low_bound,RSI_vol_window,RSI_vol_q_high,RSI_vol_q_low,RSI_first_candle,RSI_second_candle,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
70,STOCH_RSI,14,35,48,0.25,0.90,0.8,0.5,9,7,3,25,136,1.622225,2.151667,84.622600,292.626667,1.3939,1.4399,1.3369,1.2390,1.2066,1.2752,1.3691,1.4017,1.3818,1.5680,1.6310,1.6255,1.5789,1.8350,1.7924,1.7912,1.7801,1.8186,1.8375,1.8839,1.8804,1.9310,1.9800,1.9558,0.22,0.43,0.64,0.83,1.03,1.23,1.43,1.62,1.80,1.99,2.18,2.36,2.52,2.67,2.80,2.90,2.99,3.06,3.11,3.14,3.13,3.15,3.19,3.22
30,STOCH_RSI,14,35,48,0.15,0.70,0.8,0.5,9,7,3,25,136,1.622225,2.151667,84.622600,292.626667,1.3939,1.4399,1.3369,1.2390,1.2066,1.2752,1.3691,1.4017,1.3818,1.5680,1.6310,1.6255,1.5789,1.8350,1.7924,1.7912,1.7801,1.8186,1.8375,1.8839,1.8804,1.9310,1.9800,1.9558,0.22,0.43,0.64,0.83,1.03,1.23,1.43,1.62,1.80,1.99,2.18,2.36,2.52,2.67,2.80,2.90,2.99,3.06,3.11,3.14,3.13,3.15,3.19,3.22
12,STOCH_RSI,14,35,48,0.10,0.70,0.8,0.5,9,7,3,25,136,1.622225,2.151667,84.622600,292.626667,1.3939,1.4399,1.3369,1.2390,1.2066,1.2752,1.3691,1.4017,1.3818,1.5680,1.6310,1.6255,1.5789,1.8350,1.7924,1.7912,1.7801,1.8186,1.8375,1.8839,1.8804,1.9310,1.9800,1.9558,0.22,0.43,0.64,0.83,1.03,1.23,1.43,1.62,1.80,1.99,2.18,2.36,2.52,2.67,2.80,2.90,2.99,3.06,3.11,3.14,3.13,3.15,3.19,3.22
14,STOCH_RSI,14,35,48,0.10,0.80,0.8,0.5,9,7,3,25,136,1.622225,2.151667,84.622600,292.626667,1.3939,1.4399,1.3369,1.2390,1.2066,1.2752,1.3691,1.4017,1.3818,1.5680,1.6310,1.6255,1.5789,1.8350,1.7924,1.7912,1.7801,1.8186,1.8375,1.8839,1.8804,1.9310,1.9800,1.9558,0.22,0.43,0.64,0.83,1.03,1.23,1.43,1.62,1.80,1.99,2.18,2.36,2.52,2.67,2.80,2.90,2.99,3.06,3.11,3.14,3.13,3.15,3.19,3.22
32,STOCH_RSI,14,35,48,0.15,0.80,0.8,0.5,9,7,3,25,136,1.622225,2.151667,84.622600,292.626667,1.3939,1.4399,1.3369,1.2390,1.2066,1.2752,1.3691,1.4017,1.3818,1.5680,1.6310,1.6255,1.5789,1.8350,1.7924,1.7912,1.7801,1.8186,1.8375,1.8839,1.8804,1.9310,1.9800,1.9558,0.22,0.43,0.64,0.83,1.03,1.23,1.43,1.62,1.80,1.99,2.18,2.36,2.52,2.67,2.80,2.90,2.99,3.06,3.11,3.14,3.13,3.15,3.19,3.22
69,STOCH_RSI,14,35,48,0.25,0.85,0.8,0.5,9,7,3,25,136,1.622225,2.151667,84.622600,292.626667,1.3939,1.4399,1.3369,1.2390,1.2066,1.2752,1.3691,1.4017,1.3818,1.5680,1.6310,1.6255,1.5789,1.8350,1.7924,1.7912,1.7801,1.8186,1.8375,1.8839,1.8804,1.9310,1.9800,1.9558,0.22,0.43,0.64,0.83,1.03,1.23,1.43,1.62,1.80,1.99,2.18,2.36,2.52,2.67,2.80,2.90,2.99,3.06,3.11,3.14,3.13,3.15,3.19,3.22
87,STOCH_RSI,14,35,48,0.30,0.85,0.8,0.5,9,7,3,25,136,1.622225,2.151667,84.622600,292.626667,1.3939,1.4399,1.3369,1.2390,1.2066,1.2752,1.3691,1.4017,1.3818,1.5680,1.6310,1.6255,1.5789,1.8350,1.7924,1.7912,1.7801,1.8186,1.8375,1.8839,1.8804,1.9310,1.9800,1.9558,0.22,0.43,0.64,0.83,1.03,1.23,1.43,1.62,1.80,1.99,2.18,2.36,2.52,2.67,2.80,2.90,2.99,3.06,3.11,3.14,3.13,3.15,3.19,3.22
71,STOCH_RSI,14,35,48,0.25,0.95,0.8,0.5,9,7,3,25,136,1.622225,2.151667,84.622600,292.626667,1.3939,1.4399,1.3369,1.2390,1.2066,1.2752,1.3691,1.4017,1.3818,1.5680,1.6310,1.6255,1.5789,1.8350,1.7924,1.7912,1.7801,1.8186,1.8375,1.8839,1.8804,1.9310,1.9800,1.9558,0.22,0.43,0.64,0.83,1.03,1.23,1.43,1.62,1.80,1.99,2.18,2.36,2.52,2.67,2.80,2.90,2.99,3.06,3.11,3.14,3.13,3.15,3.19,3.22
9,STOCH_RSI,14,35,48,0.10,0.55,0.8,0

In [23]:
# vol
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_rank', ascending=False)
stat.head(20)

,pattern,RSI_timeperiod,RSI_low_bound,RSI_vol_window,RSI_vol_q_high,RSI_vol_q_low,RSI_first_candle,RSI_second_candle,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
86,STOCH_RSI,14,35,48,0.30,0.80,0.8,0.5,9,7,3,25,136,1.622225,2.151667,84.6226,292.626667,1.3939,1.4399,1.3369,1.239,1.2066,1.2752,1.3691,1.4017,1.3818,1.568,1.631,1.6255,1.5789,1.835,1.7924,1.7912,1.7801,1.8186,1.8375,1.8839,1.8804,1.931,1.98,1.9558,0.22,0.43,0.64,0.83,1.03,1.23,1.43,1.62,1.8,1.99,2.18,2.36,2.52,2.67,2.8,2.9,2.99,3.06,3.11,3.14,3.13,3.15,3.19,3.22
70,STOCH_RSI,14,35,48,0.25,0.90,0.8,0.5,9,7,3,25,136,1.622225,2.151667,84.6226,292.626667,1.3939,1.4399,1.3369,1.239,1.2066,1.2752,1.3691,1.4017,1.3818,1.568,1.631,1.6255,1.5789,1.835,1.7924,1.7912,1.7801,1.8186,1.8375,1.8839,1.8804,1.931,1.98,1.9558,0.22,0.43,0.64,0.83,1.03,1.23,1.43,1.62,1.8,1.99,2.18,2.36,2.52,2.67,2.8,2.9,2.99,3.06,3.11,3.14,3.13,3.15,3.19,3.22
31,STOCH_RSI,14,35,48,0.15,0.75,0.8,0.5,9,7,3,25,136,1.622225,2.151667,84.6226,292.626667,1.3939,1.4399,1.3369,1.239,1.2066,1.2752,1.3691,1.4017,1.3818,1.568,1.631,1.6255,1.5789,1.835,1.7924,1.7912,1.7801,1.8186,1.8375,1.8839,1.8804,1.931,1.98,1.9558,0.22,0.43,0.64,0.83,1.03,1.23,1.43,1.62,1.8,1.99,2.18,2.36,2.52,2.67,2.8,2.9,2.99,3.06,3.11,3.14,3.13,3.15,3.19,3.22
34,STOCH_RSI,14,35,48,0.15,0.90,0.8,0.5,9,7,3,25,136,1.622225,2.151667,84.6226,292.626667,1.3939,1.4399,1.3369,1.239,1.2066,1.2752,1.3691,1.4017,1.3818,1.568,1.631,1.6255,1.5789,1.835,1.7924,1.7912,1.7801,1.8186,1.8375,1.8839,1.8804,1.931,1.98,1.9558,0.22,0.43,0.64,0.83,1.03,1.23,1.43,1.62,1.8,1.99,2.18,2.36,2.52,2.67,2.8,2.9,2.99,3.06,3.11,3.14,3.13,3.15,3.19,3.22
87,STOCH_RSI,14,35,48,0.30,0.85,0.8,0.5,9,7,3,25,136,1.622225,2.151667,84.6226,292.626667,1.3939,1.4399,1.3369,1.239,1.2066,1.2752,1.3691,1.4017,1.3818,1.568,1.631,1.6255,1.5789,1.835,1.7924,1.7912,1.7801,1.8186,1.8375,1.8839,1.8804,1.931,1.98,1.9558,0.22,0.43,0.64,0.83,1.03,1.23,1.43,1.62,1.8,1.99,2.18,2.36,2.52,2.67,2.8,2.9,2.99,3.06,3.11,3.14,3.13,3.15,3.19,3.22
125,STOCH_RSI,14,35,48,0.40,0.95,0.8,0.5,9,7,3,25,136,1.622225,2.151667,84.6226,292.626667,1.3939,1.4399,1.3369,1.239,1.2066,1.2752,1.3691,1.4017,1.3818,1.568,1.631,1.6255,1.5789,1.835,1.7924,1.7912,1.7801,1.8186,1.8375,1.8839,1.8804,1.931,1.98,1.9558,0.22,0.43,0.64,0.83,1.03,1.23,1.43,1.62,1.8,1.99,2.18,2.36,2.52,2.67,2.8,2.9,2.99,3.06,3.11,3.14,3.13,3.15,3.19,3.22
13,STOCH_RSI,14,35,48,0.10,0.75,0.8,0.5,9,7,3,25,136,1.622225,2.151667,84.6226,292.626667,1.3939,1.4399,1.3369,1.239,1.2066,1.2752,1.3691,1.4017,1.3818,1.568,1.631,1.6255,1.5789,1.835,1.7924,1.7912,1.7801,1.8186,1.8375,1.8839,1.8804,1.931,1.98,1.9558,0.22,0.43,0.64,0.83,1.03,1.23,1.43,1.62,1.8,1.99,2.18,2.36,2.52,2.67,2.8,2.9,2.99,3.06,3.11,3.14,3.13,3.15,3.19,3.22
17,STOCH_RSI,14,35,48,0.10,0.95,0.8,0.5,9,7,3,25,136,1.622225,2.151667,84.6226,292.626667,1.3939,1.4399,1.3369,1.239,1.2066,1.2752,1.3691,1.4017,1.3818,1.568,1.631,1.6255,1.5789,1.835,1.7924,1.7912,1.7801,1.8186,1.8375,1.8839,1.8804,1.931,1.98,1.9558,0.22,0.43,0.64,0.83,1.03,1.23,1.43,1.62,1.8,1.99,2.18,2.36,2.52,2.67,2.8,2.9,2.99,3.06,3.11,3.14,3.13,3.15,3.19,3.22
85,STOCH_RSI,14,35,48,0.30,0.75,0.8,0.5,9,7,3,25,136,1.622225,2.151667,84.6226,292.626667,1.3939,1.4399,1.3369,1.239,1.2066,1.2752,

In [2]:
# vol_window
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_rank', ascending=False)
stat.head(20)

,pattern,RSI_timeperiod,RSI_low_bound,RSI_vol_window,RSI_vol_q_high,RSI_vol_q_low,RSI_first_candle,RSI_second_candle,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
0,STOCH_RSI,14,35,6,0.2,0.8,0.8,0.5,9,7,3,25,135,1.631792,2.175833,85.291875,293.737500,1.4310,1.4670,1.3483,1.2516,1.2170,1.2881,1.3727,1.4045,1.3867,1.5755,1.6351,1.6294,1.5824,1.8395,1.8021,1.8040,1.7936,1.8342,1.8511,1.8927,1.8895,1.9310,1.9801,1.9559,0.22,0.43,0.64,0.84,1.04,1.25,1.44,1.63,1.82,2.01,2.20,2.38,2.54,2.69,2.83,2.94,3.03,3.10,3.15,3.18,3.17,3.19,3.23,3.27
2,STOCH_RSI,14,35,24,0.2,0.8,0.8,0.5,9,7,3,25,136,1.622225,2.151667,84.622600,292.626667,1.3939,1.4399,1.3369,1.2390,1.2066,1.2752,1.3691,1.4017,1.3818,1.5680,1.6310,1.6255,1.5789,1.8350,1.7924,1.7912,1.7801,1.8186,1.8375,1.8839,1.8804,1.9310,1.9800,1.9558,0.22,0.43,0.64,0.83,1.03,1.23,1.43,1.62,1.80,1.99,2.18,2.36,2.52,2.67,2.80,2.90,2.99,3.06,3.11,3.14,3.13,3.15,3.19,3.22
3,STOCH_RSI,14,35,48,0.2,0.8,0.8,0.5,9,7,3,25,136,1.622225,2.151667,84.622600,292.626667,1.3939,1.4399,1.3369,1.2390,1.2066,1.2752,1.3691,1.4017,1.3818,1.5680,1.6310,1.6255,1.5789,1.8350,1.7924,1.7912,1.7801,1.8186,1.8375,1.8839,1.8804,1.9310,1.9800,1.9558,0.22,0.43,0.64,0.83,1.03,1.23,1.43,1.62,1.80,1.99,2.18,2.36,2.52,2.67,2.80,2.90,2.99,3.06,3.11,3.14,3.13,3.15,3.19,3.22
4,STOCH_RSI,14,35,72,0.2,0.8,0.8,0.5,9,7,3,25,136,1.622225,2.151667,84.622600,292.626667,1.3939,1.4399,1.3369,1.2390,1.2066,1.2752,1.3691,1.4017,1.3818,1.5680,1.6310,1.6255,1.5789,1.8350,1.7924,1.7912,1.7801,1.8186,1.8375,1.8839,1.8804,1.9310,1.9800,1.9558,0.22,0.43,0.64,0.83,1.03,1.23,1.43,1.62,1.80,1.99,2.18,2.36,2.52,2.67,2.80,2.90,2.99,3.06,3.11,3.14,3.13,3.15,3.19,3.22
5,STOCH_RSI,14,35,96,0.2,0.8,0.8,0.5,9,7,3,25,136,1.622225,2.151667,84.622600,292.626667,1.3939,1.4399,1.3369,1.2390,1.2066,1.2752,1.3691,1.4017,1.3818,1.5680,1.6310,1.6255,1.5789,1.8350,1.7924,1.7912,1.7801,1.8186,1.8375,1.8839,1.8804,1.9310,1.9800,1.9558,0.22,0.43,0.64,0.83,1.03,1.23,1.43,1.62,1.80,1.99,2.18,2.36,2.52,2.67,2.80,2.90,2.99,3.06,3.11,3.14,3.13,3.15,3.19,3.22
1,STOCH_RSI,14,35,12,0.2,0.8,0.8,0.5,9,7,3,25,130,1.721387,2.182917,93.780375,283.779167,1.4720,1.5293,1.4137,1.3213,1.2796,1.3531,1.4525,1.4815,1.4615,1.6667,1.7306,1.7188,1.6691,1.9441,1.9001,1.9022,1.8918,1.9346,1.9537,2.0027,1.9992,2.0543,2.1050,2.0759,0.22,0.43,0.64,0.84,1.04,1.24,1.44,1.63,1.82,2.01,2.20,2.38,2.55,2.70,2.84,2.95,3.04,3.11,3.16,3.19,3.19,3.21,3.26,3.30


In [2]:
# trend
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_rank', ascending=False)
stat.head(20)

,pattern,RSI_timeperiod,RSI_low_bound,RSI_vol_window,RSI_vol_q_high,RSI_vol_q_low,RSI_first_candle,RSI_second_candle,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
2,STOCH_RSI_Trend,14,35,48,0.2,0.8,0.8,0.5,9,7,3,25,6,0,135,1.655513,2.169583,88.494188,292.893750,1.4281,1.4666,1.3763,1.2798,1.2407,1.3095,1.4038,1.4343,1.4130,1.6013,1.6645,1.6602,1.6112,1.8713,1.8261,1.8242,1.8123,1.8512,1.8701,1.9159,1.9111,1.9624,2.0119,1.9865,0.22,0.43,0.64,0.84,1.04,1.24,1.44,1.63,1.82,2.01,2.20,2.37,2.54,2.69,2.82,2.93,3.02,3.09,3.14,3.16,3.15,3.18,3.22,3.25
3,STOCH_RSI_Trend,14,35,48,0.2,0.8,0.8,0.5,9,7,3,25,8,0,135,1.655513,2.169583,88.494188,292.893750,1.4281,1.4666,1.3763,1.2798,1.2407,1.3095,1.4038,1.4343,1.4130,1.6013,1.6645,1.6602,1.6112,1.8713,1.8261,1.8242,1.8123,1.8512,1.8701,1.9159,1.9111,1.9624,2.0119,1.9865,0.22,0.43,0.64,0.84,1.04,1.24,1.44,1.63,1.82,2.01,2.20,2.37,2.54,2.69,2.82,2.93,3.02,3.09,3.14,3.16,3.15,3.18,3.22,3.25
4,STOCH_RSI_Trend,14,35,48,0.2,0.8,0.8,0.5,9,7,3,25,10,0,134,1.656183,2.178750,87.928567,291.952500,1.4164,1.4629,1.3701,1.2743,1.2316,1.3010,1.3960,1.4272,1.4062,1.5952,1.6587,1.6548,1.6059,1.8671,1.8370,1.8349,1.8227,1.8618,1.8808,1.9270,1.9220,1.9737,2.0235,1.9976,0.22,0.44,0.65,0.84,1.04,1.25,1.45,1.64,1.82,2.01,2.21,2.38,2.55,2.70,2.83,2.94,3.03,3.10,3.15,3.18,3.17,3.19,3.23,3.27
5,STOCH_RSI_Trend,14,35,48,0.2,0.8,0.8,0.5,9,7,3,25,12,0,133,1.695183,2.191250,92.459383,291.436250,1.4358,1.4781,1.3817,1.2917,1.2665,1.3379,1.4333,1.4620,1.4387,1.6299,1.6935,1.6875,1.6363,1.9013,1.8911,1.8881,1.8744,1.9141,1.9331,1.9803,1.9744,2.0273,2.0777,2.0497,0.22,0.44,0.65,0.85,1.05,1.25,1.45,1.64,1.83,2.02,2.22,2.40,2.56,2.71,2.85,2.96,3.05,3.12,3.17,3.20,3.19,3.21,3.26,3.29
6,STOCH_RSI_Trend,14,35,48,0.2,0.8,0.8,0.5,9,7,3,25,14,0,131,1.718071,2.207917,94.067279,289.237083,1.4366,1.4864,1.3994,1.3019,1.2749,1.3444,1.4448,1.4761,1.4516,1.6451,1.7176,1.7101,1.6569,1.9365,1.9258,1.9222,1.9074,1.9476,1.9667,2.0148,2.0082,2.0621,2.1130,2.0836,0.22,0.44,0.65,0.85,1.05,1.26,1.46,1.65,1.84,2.04,2.23,2.41,2.58,2.73,2.87,2.98,3.08,3.15,3.20,3.23,3.22,3.24,3.29,3.32
1,STOCH_RSI_Trend,14,35,48,0.2,0.8,0.8,0.5,9,7,3,25,4,0,133,1.674012,2.170417,89.643662,288.665417,1.4464,1.4699,1.3774,1.2756,1.2357,1.3057,1.4027,1.4390,1.4171,1.6213,1.6852,1.6809,1.6395,1.9053,1.8572,1.8549,1.8418,1.8808,1.8997,1.9462,1.9409,1.9932,2.0433,2.0166,0.22,0.43,0.64,0.84,1.04,1.24,1.44,1.62,1.81,2.00,2.19,2.37,2.53,2.69,2.82,2.93,3.02,3.10,3.14,3.17,3.16,3.19,3.23,3.27
7,STOCH_RSI_Trend,14,35,48,0.2,0.8,0.8,0.5,9,7,3,25,16,0,129,1.747246,2.232083,96.394712,287.938750,1.4299,1.4855,1.3992,1.3186,1.2959,1.3693,1.4778,1.5187,1.4909,1.6911,1.7644,1.7538,1.6972,1.9896,1.9679,1.9572,1.9413,1.9806,1.9909,2.0402,2.0338,2.0891,2.1410,2.1100,0.22,0.45,0.66,0.86,1.07,1.27,1.48,1.67,1.86,2.06,2.26,2.44,2.61,2.76,2.90,3.01,3.11,3.18,3.23,3.26,3.25,3.28,3.32,3.36
8,STOCH_RSI_Trend,14,35,48,0.2,0.8,0.8,0.5,9,7,3,25,18,0,128,1.724125,2.229167,92.688000,285.333333,1.4241,1.4796,1.3940,1.3238,1.2856,1.3554,1.4655,1.5000,1.4706,1.6622,1.7254,1.7172,1.6619,1.9569,1.9362,1.9259,1.9106,1.9505,1.9612,2.0109,2.0049,2.0606,2.1132,2.0828,0.22,0.45,0.66,0.86,1.07,1.27,1.48,1.67,1.86,2.06,2.26,2.44,2.

# Research parameter influence

In [16]:
param = 'Trend_timeperiod'

res = stat.groupby([param]).agg({'e_ratio_avg': 'mean',
                                 'pct_price_diff_avg': 'mean',
                                 'e_ratio_rank': 'mean', 
                                 'price_rank': 'mean', 
                                 'forecasts_num': 'sum'}).sort_values(param).reset_index()
res

,Trend_timeperiod,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank,forecasts_num
0,2,3.236211,1.144583,86.068997,44.164444,114
1,3,5.849584,1.552812,62.963132,19.165312,56
2,4,6.061108,1.460000,53.063822,15.588472,32
3,5,6.842036,1.795556,47.689653,14.873611,25
4,6,5.730101,1.770000,37.715247,13.606111,26
5,8,5.441139,1.488750,57.024707,21.980104,61


# Save new config data to config file

In [3]:
from config_updater import ConfigUpdater

ttype = 'buy'
pattern = ['STOCH', 'RSI']
work_timeframe = '15m'
higher_timeframe = '1h'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {
                'RSI': {
                        'timeperiod': [14], 
                        'low_bound': [35],
                        'vol_window': [48],
                        'vol_q_high': [0.2],
                        'vol_q_low': [0.8],
                        'first_candle': [0.8],
                        'second_candle': [0.5]
                       },
                'STOCH': {
                          'fastk_period': [9],
                          'slowk_period': [7],
                          'slowd_period': [3], 
                          'low_bound': [25]
                        },
                'Trend': {
                          'timeperiod': [48],
                          'low_bound': [0]
                        }
             }
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)

# Check global statistics

In [2]:
import numpy as np
import pandas as pd
from glob import glob

ttype = 'buy'
pattern = ['STOCH', 'RSI']
work_timeframe = '5m'
higher_timeframe = '1h'
stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
stat = None

for sl in stat_list:
    tmp = pd.read_pickle(sl)
    if stat is None:
        stat = tmp.copy()
    else:
        stat = pd.concat([stat, tmp])
        
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
        
total_stat = stat.groupby(['RSI_timeperiod', 
                           'RSI_low_bound', 
                           'STOCH_fastk_period', 
                           'STOCH_slowk_period', 
                           'STOCH_slowd_period', 
                           'STOCH_low_bound']).agg({'pct_right_forecast_avg': 'mean',
                                                    'pct_price_diff_avg': 'mean',
                                                    'forecast_rank': 'mean', 
                                                    'price_rank': 'mean', 
                                                    'forecasts_num': 'sum'}).sort_values('forecast_rank', 
                                                                                         ascending=False)
total_stat.head(20)

pct_right_forecast_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                           
16             30            5                  4                  3                  10                            75.759735   
                                                                   5                  10                            83.794091   
               35            11                 3                  9                  20                            66.518698   
               25            7                  3                  5                  10                            88.219697   
12             20            7                  3                  5                  10                            86.237538   
               30            11                 3                  9                  25                            67.022344   
               25            5                  4                  5                  10                            79.443447   
14             30            5                  4                  5                  10                            72.603182   
12             20            7                  4                  5                  10                            59.343523   
16             25            9                  3                  5                  10                            73.011894   
                             7                  3                  7                  10                            56.439394   
12             20            7                  3                  7                  10                            65.530303   
16             25            7                  4                  5                  10                            54.286970   
                             5                  3                  7                  10                            58.333333   
12             20            5                  3                  7                  10                            58.333333   
                             9                  3                  5                  10                            73.201098   
               30            5                  4                  5                  10                            69.466932   
16             25            7                  2                  5                  10                            77.102348   
               35            5                  4                  5                  10                            69.892652   
14             20            5                  5                  5                  10                            75.757576   

                                                                                                       pct_price_diff_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                       
16             30            5                  4                  3                  10                         0.843144   
                                                                   5                  10                         1.436705   
               35            11                 3                  9                  20                         0.455156   
               25            7                  3                  5                  10                         1.885038   
12             20            7                  3                  5                  10                         1.803295   
               30            11                 3                  9                  25                         0.418698   
               25            5                  4                  5                  10                         1.366742   
14             30            5                  4                  5                  10                         0.644697   
12            